 Login to your azure account az login - device

In [ ]:
pip  install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 -U

In [ ]:
! pip install olive-ai[auto-opt] -U

In [ ]:
pip install onnxruntime-genai==0.7.0 --pre

In [ ]:
! pip install onnxruntime

In [ ]:
! pip install bitsandbytes

In [ ]:
! pip install transformers==4.49.0 -U

In [ ]:
! pip install azure-ai-ml -U  

In [ ]:
! pip install marshmallow==3.23.2 -U   

In [ ]:
! pip install tf-keras

In [ ]:
! pip install numpy==1.23.5 -U

In [ ]:
! pip install peft

In [ ]:
! olive finetune \
    --method lora \
    --model_name_or_path  azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 \
    --trust_remote_code \
    --data_name json \
    --data_files ./data/train_data.jsonl \
    --text_template "<|user|>{Question}<|end|><|assistant|>{Answer}<|end|>" \
    --max_steps 100 \
    --output_path models/phi-4-mini/ft \
    --target_modules "q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj" \
    --log_level 1

In [ ]:
! pip install onnxruntime-genai==0.7.0 --pre

In [ ]:
! olive auto-opt \
    --model_name_or_path  azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 \
    --adapter_path models/phi-4-mini/ft/adapter \
    --device cpu \
    --provider CPUExecutionProvider \
    --use_model_builder \
    --precision int4 \
    --output_path models/phi-4-mini/onnx \
    --log_level 1